In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def load_movielens_data():
    user_columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
    item_columns = ['movie_id', 'title','release_date', 'video_release_date', 'IMDb_URL', 
                    'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 
                    'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 
                    'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    rating_columns = ['user_id','movie_id', 'rating', 'timestamp']
    try:
        users = pd.read_csv('ml-100k/u.user', sep='|', names=user_columns)
        items = pd.read_csv('ml-100k/u.item', sep='|', names=item_columns, encoding='latin-1')
        ratings = pd.read_csv('ml-100k/u.data', sep='\t', names=rating_columns)
        return users, items, ratings
    except FileNotFoundError:
        print("Please ensure that the ml-100k dataset file is in the correct path")
        return None, None, None

In [3]:
def create_utility_matrix(ratings):
    utility_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
    return utility_matrix

In [4]:
def calculate_similarity(utility_matrix):
    user_means = utility_matrix.mean(axis=1)
    centered_matrix = utility_matrix.sub(user_means, axis=0)
    centered_matrix_filled = centered_matrix.fillna(0)
    similarity_matrix = cosine_similarity(centered_matrix_filled)
    similarity_df = pd.DataFrame(similarity_matrix, 
                                 index=utility_matrix.index, 
                                 columns=utility_matrix.index)
    
    return similarity_df, centered_matrix, user_means

In [5]:
def build_user_profile(user_id, centered_matrix, user_means):
    user_profile = centered_matrix.loc[user_id].to_dict()
    user_profile['average_rating'] = user_means[user_id]
    return user_profile

In [6]:
def calculate_similarity_distance(user_profile, item_id, utility_matrix):
    item_vector = utility_matrix.loc[:, item_id].dropna()
    user_vector = pd.Series(user_profile).reindex(item_vector.index, fill_value=0)
    if np.isnan(user_vector).any() or np.isnan(item_vector).any():
        raise ValueError("NaN value still exists in the processed vector, please further check the data processing logic")
    
    similarity = cosine_similarity([user_vector], [item_vector])[0][0]
    distance = np.linalg.norm(user_vector - item_vector)
    
    return similarity, distance

In [7]:
def main():
    users, items, ratings = load_movielens_data()
    if users is None:
        return
    utility_matrix = create_utility_matrix(ratings)
    similarity_df, centered_matrix, user_means = calculate_similarity(utility_matrix)
    
    target_user_id_1 = 200
    target_user_id_2 = 15
    target_item_id = 95
    
    user_profile_1 = build_user_profile(target_user_id_1, centered_matrix, user_means)
    user_profile_2 = build_user_profile(target_user_id_2, centered_matrix, user_means)
    
    similarity_1, distance_1 = calculate_similarity_distance(user_profile_1, target_item_id, utility_matrix)
    similarity_2, distance_2 = calculate_similarity_distance(user_profile_2, target_item_id, utility_matrix)
    
    print(f"Cosine similarity between User {target_user_id_1} and item {target_item_id}: {similarity_1:.2f}")
    print(f"Distance between User {target_user_id_1} and Item {target_item_id}: {distance_1:.2f}")
    print(f"Cosine similarity between User {target_user_id_2} and Item {target_item_id}: {similarity_2:.2f}")
    print(f"Distance between User {target_user_id_2} and Item {target_item_id}: {distance_2:.2f}")
    
    # 推荐决策
    if similarity_1 > similarity_2:
        print(f"The recommendation system will suggest recommending the movie {target_item_id} to the user {target_user_id_1}")
    else:
        print(f"The recommendation system will suggest recommending the movie {target_item_id} to the user {target_user_id_2}")

In [8]:
if __name__ == "__main__":
    main()

Cosine similarity between User 200 and item 95: 0.05
Distance between User 200 and Item 95: 77.17
Cosine similarity between User 15 and Item 95: 0.10
Distance between User 15 and Item 95: 63.17
The recommendation system will suggest recommending the movie 95 to the user 15
